# Load library

In [7]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

# SentenceTokenizer Class

In [8]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.okt = Okt()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
        ,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",'로써',
        '해당']
    def url2sentences(self, url):
        article = Article(url, language='ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)
        print(article.text)
        print('*'*120)

        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
            return sentences

    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences

    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence != '':
              nouns.append(' '.join([noun for noun in self.okt.nouns(str(sentence))
            if noun not in self.stopwords and len(noun) > 1]))
        return nouns


# GrapMatrix Class

In [9]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
        
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence

    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

# Rank Class

In [10]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1

        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

# TextRank Class

In [11]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)

    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)

        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])

        return summary

    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)

        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)

        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords


# Main

In [12]:
url = input()
textrank = TextRank(url)
for row in textrank.summarize(5):
    print(row)
    print()
print('keywords :',textrank.keywords())

이리하고 저리하고..

아무리 열어보려고 해도 꽉 닫혀버린 너!!

생활팁으로 알아보는 간단한 도장 뚜껑 여는법.. 알고보니 참 쉽죠~

























인터넷 뱅킹을 주로하다보니 은행에 갈 일이.. 일년에 손에 꼽을 정도라고나 할까요?





그런데... 이번에 은행 갈일이 생겨서.. 이것저것 챙겨보게 되면서..

도장을 꺼내보았지요.





몇 달.. 아니 몇 년만에 마주하는 것만 같은 내 도장..

그동안 잘 있었나? 하고 테스트할겸..뚜껑을 열어보려는데............





열리지가 않는다..

꿈쩍도 하지 않는다...





이...이럴수가..





내 손힘이 약해서 또 못여는구나.. 하며 좌절...





​

​













랑~~~~ 이리와서 열어보아요~!!

이럴때는 신랑을 부르는게 최고지요~!





평소에도 뚜껑같은 건 잘 못여는저라서..

신랑은 도장가지고 씨름하는 저를 보더니 역시나~ 하는 표정을 지으면서 오는데..





훗...

자기도 못 열면서!!





어찌나 꽉 닫혀 있는건지.. 꿈쩍도 하지 않는 도장 뚜껑!!





너를 어쩌면 좋으란 말이던가..ㅜ^ㅜ





​









​

그래서 급 도장 뚜껑 여는 법을 찾아 헤매이는데~~

아....

어째 팁들이.. 죄... 나무도장이란 말이렷다!!





내꺼는.. 아크릴 플라스틱이란 말인데..ㅜ^ㅜ





나무도장처럼.. 뜨거운 물에 넣는 것으로는.. 아니 된단 말이죠.. 흑흑흑













* 참고로 나무도장의 경우 컵에 뜨거운 물을 담은 후

그 안에 도장을 5분정도 담가두면 쉽게 열 수 있다고 합니다.

















하지만!! 저도 해결했답니다~!!

저같은 플라틱 도장은요.. 차가운 냉동실로 고고씽~!!!





10분정도 넣어두었다가 꺼내보세요.

​

​













그러면..

그렇게 꽉! 껴서는.. 신랑이 열려고 해도 열리지 않던 도장 뚜껑이..

저